In [11]:
from transformers import BertTokenizer, BertModel
import torch
from torch import nn

In [12]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained("dbmdz/bert-base-turkish-cased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [24]:
# load model and tokenizer with loading state dict
model_offensive = torch.load('./models/bert_model_is_offensive.pt')
model_target = torch.load('./models/bert_model_target.pt')
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

In [15]:
def predict(model, sentence):
    
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
    
        if use_cuda:
            model = model.cuda()
    
        with torch.no_grad():
    
            input_id = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
            mask = input_id['attention_mask'].to(device)
            input_id = input_id['input_ids'].squeeze(1).to(device)
    
            output = model(input_id, mask)
    
            return output.argmax(dim=1).item()

In [35]:
text = 'Beğenmiyorsan yallah arabistana git'
prediction_1 = predict(model_offensive, text)
if prediction_1 == 0:
    print("Not offensive")
else:
    prediction_2 = predict(model_target, text)
    if prediction_2 == 0:
        print("Offensive, Insult")
    elif prediction_2 == 1:
        print("Offensive, Racist")
    elif prediction_2 == 2:
        print("Offensive, Sexist")
    elif prediction_2 == 3:
        print("Offensive, Profanity")

Offensive, Racist
